In [ ]:
%pip install plotly
%pip install nbformat

In [2]:
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.structs import TopicPartition

import plotly.express as px
import json
import pandas as pd

In [25]:
from kafka import KafkaConsumer

# auto_offset_reset & group_id are CRUCIAL for it to work. No idea why
consumer = KafkaConsumer('energy_api', consumer_timeout_ms=10000, bootstrap_servers = 'localhost:9092', auto_offset_reset='earliest', group_id=None)

dfs = []

partitions = consumer.partitions_for_topic('energy_api')

for p in partitions:
    topic_partition = TopicPartition('energy_api', p)
    # Seek offset 0
    # consumer.seek(partition=topic_partition, offset=0)
    for msg in consumer:
        try:
            dict = json.loads(msg.value)
            df = pd.DataFrame.from_records(dict)
            dfs.append(df)
            
        except:
            print(msg.value[:20])
            continue
df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.
df

b'TEST'
b'TEST2'


,Minutes5UTC,Minutes5DK,PriceArea,ProductionLt100MW,ProductionGe100MW,OffshoreWindPower,OnshoreWindPower,SolarPower,ExchangeGreatBelt,ExchangeGermany,ExchangeNetherlands,ExchangeNorway,ExchangeSweden,BornholmSE4
0,2023-02-06T09:20:00,2023-02-06T10:20:00,DK1,594.280029,1022.679993,-7.280000,28.850000,218.990005,-354.000000,-568.909973,700.380005,1107.680054,236.000000,NaN
1,2023-02-06T09:20:00,2023-02-06T10:20:00,DK2,195.460007,548.739990,31.840000,25.070000,77.379997,354.000000,-421.899994,NaN,NaN,974.669983,32.049999
2,2023-02-06T09:25:00,2023-02-06T10:25:00,DK1,592.340027,1012.020020,-6.270000,22.940001,234.770004,-354.000000,-558.880005,700.380005,1107.339966,236.000000,NaN
3,2023-02-06T09:25:00,2023-02-06T10:25:00,DK2,198.199997,551.010010,24.440001,24.740000,76.830002,354.000000,-421.799988,NaN,NaN,1017.760010,33.689999
4,2023-02-06T09:30:00,2023-02-06T10:30:00,DK1,597.530029,993.780029,-2.800000,20.780001,235.080002,-354.000000,-538.559998,700.380005,1107.770020,236.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2023-02-06T09:40:00,2023-02-06T10:40:00,DK2,199.220001,548.549988,13.660000,25.559999,86.760002,354.000000,-422.190002,NaN,NaN,980.859985,32.430000
114,2023-02-06T09:45:00,2023-02-06T10:45:00,DK1,585.700012,976.580017,2.040000,17.700001,251.490005,-354.079987,-564.239990,700.380005,1107.530029,236.000000,NaN
115,2023-02-06T09:45:00,2023-02-06T10:45:00,DK2,194.229996,548.830017,9.590000,26.020000,87.750000,354.079987,-421.250000,NaN,NaN,989.140015,32.840000
116,2023-02-06T09:50:00,2023-02-06T10:50:00,DK1,589.419983,972.770020,3.370000,19.629999,257.420013,-287.910004,-706.950012,700.380005,1107.819946,312.000000,NaN


In [26]:
df["Minutes5DK"] = pd.to_datetime(df["Minutes5DK"])
df["Minutes5UTC"] = pd.to_datetime(df["Minutes5UTC"])

In [30]:
avg_by_minute = df.groupby(df.Minutes5DK).mean()
avg_by_minute.head()

/tmp/ipykernel_9696/1471977883.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,ProductionLt100MW,ProductionGe100MW,OffshoreWindPower,OnshoreWindPower,SolarPower,ExchangeGreatBelt,ExchangeGermany,ExchangeNetherlands,ExchangeNorway,ExchangeSweden,BornholmSE4
Minutes5DK,,,,,,,,,,,
2023-02-02 19:35:00,408.099999,792.309998,246.910005,406.589989,0.095,0.0,-1340.479980,700.039978,1109.449951,778.239990,26.549999
2023-02-02 19:40:00,407.565010,788.039978,247.250002,411.050003,0.090,0.0,-1350.304993,700.039978,1110.329956,784.709992,26.620001
2023-02-02 19:45:00,408.435013,776.605011,260.690005,419.334999,0.095,0.0,-1371.515015,700.039978,1109.199951,771.200012,26.650000
2023-02-02 19:50:00,403.229996,766.269989,272.569985,428.825012,0.100,0.0,-1403.049988,700.039978,1104.819946,784.600006,26.719999
2023-02-03 18:10:00,256.095009,598.299988,960.029999,998.109970,2.150,0.0,63.964996,-700.109985,-813.000000,446.144989,-1.250000


In [31]:
px.line(data_frame = avg_by_minute, x=avg_by_minute.index, y=["OffshoreWindPower", "OnshoreWindPower", "SolarPower"])

In [32]:
px.line(data_frame = avg_by_minute, x=avg_by_minute.index, y=["ExchangeGermany", "ExchangeNetherlands", "ExchangeNorway", "ExchangeSweden"])
